In [10]:
%cd ..

D:\ai\GNN


C:\Users\kkire\anaconda3\envs\dl\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [12]:
%ls

 Volume in drive D is DATA
 Volume Serial Number is 4096-85EF

 Directory of D:\ai\GNN

10-06-2024  12:18    <DIR>          .
07-06-2024  23:38    <DIR>          ..
10-06-2024  12:26    <DIR>          experiments
08-06-2024  18:03                 0 main.py
08-06-2024  18:11    <DIR>          models
10-06-2024  11:47             2,470 reqirements.txt
08-06-2024  23:49    <DIR>          utils
               2 File(s)          2,470 bytes
               5 Dir(s)  973,255,028,736 bytes free


In [65]:
import torch
from torch import nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from models.layers import gLayer
from models.model import GNN

import numpy as np

In [92]:

class SimpleGNN(nn.Module):
    def __init__(self, n_layers=4, emb_dim=50, edge_dim=4, in_dim=10, out_dim=1):

        super().__init__()


        self.lin_in = Linear(in_dim, emb_dim)   

        self.convs = torch.nn.ModuleList()

        for i in range(n_layers):
            self.convs.append(gLayer(emb_dim, edge_dim))
        
        self.pool = global_mean_pool
        
        self.lin_pred = Linear(emb_dim, out_dim)
    

    def forward(self, data):
        
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        
        h = self.lin_in(x)
        
        for conv in self.convs:
            h = h + conv(h, edge_index, edge_attr)
        
        print(h.shape)
        h_graph = self.pool(h, data.batch)
        
        print(h_graph.shape)
        
        out = self.lin_pred(h_graph)
        print(out.shape)
        
        return out.view(-1)


In [57]:
data_ = {'x':torch.randn(5, 10), 'edge_index':torch.randint(0, 5, (2, 6)), 'edge_attr':torch.randn(6, 4)}

In [58]:
data_['x']
data_['edge_index'].view(1, 2, 6)
# data_['edge_attr']

tensor([[[0, 3, 3, 3, 4, 1],
         [0, 1, 3, 2, 4, 3]]])

In [94]:
model = SimpleGNN()

print(type(model).__name__)
print(model)

total_param = 0
for param in model.parameters():
    total_param += np.prod(list(param.data.size()))
print(f'Total parameters: {total_param}')

SimpleGNN
SimpleGNN(
  (lin_in): Linear(in_features=10, out_features=50, bias=True)
  (convs): ModuleList(
    (0-3): 4 x gLayer(emb_dim=50, aggr=add)
  )
  (lin_pred): Linear(in_features=50, out_features=1, bias=True)
)
Total parameters: 63801


In [78]:
data = Data(x=data_['x'], edge_index=data_['edge_index'], edge_attr=data_['edge_attr'])

# Create a DataLoader
data_list = [data] * 10  # Creating multiple copies to simulate a dataset
dataloader = DataLoader(data_list, batch_size=4, shuffle=True)

In [79]:
it = iter(dataloader)
b1, b2 = next(it), next(it)
print(b1[0])

Data(x=[5, 10], edge_index=[2, 6], edge_attr=[6, 4])


In [90]:
print(b1)

DataBatch(x=[20, 10], edge_index=[2, 24], edge_attr=[24, 4], batch=[20], ptr=[5])


In [96]:
for d in dataloader:
    print(d)
    output = model(d)
    break

DataBatch(x=[20, 10], edge_index=[2, 24], edge_attr=[24, 4], batch=[20], ptr=[5])
torch.Size([20, 50])
torch.Size([4, 50])
torch.Size([4, 1])


In [97]:
print(output)

tensor([-1.8825, -1.8825, -1.8825, -1.8825], grad_fn=<ViewBackward0>)
